In [73]:
import pandas as pd
import matplotlib.pyplot as plt

# ModuleNotFoundError: No module named 'config_copy' (or 'config')
# from config_copy import geoapify_key
# Hard code in geoapify_key for now:
geoapify_key  = 'ba9029bb005d4d71a53d7b8740cb6ea6'

import json
import requests
from pprint import pprint
import hvplot.pandas

In [74]:
# Priya's API pull for data.ny.gov - ./Resources/Annual_Pop_and_Density_by_County_beginning_1970.csv

In [75]:
# Read Data.ny.gov data in to popdenscounty dataframe from CSV

popdenscounty=pd.read_csv("./Resources/Annual_Pop_and_Density_by_County_beginning_1970.csv")
print(popdenscounty)

      FIPS Code      Geography  Year                     Program Type  \
0         36001  Albany County  1970  Intercensal Population Estimate   
1         36001  Albany County  1971  Intercensal Population Estimate   
2         36001  Albany County  1972  Intercensal Population Estimate   
3         36001  Albany County  1973  Intercensal Population Estimate   
4         36001  Albany County  1974  Intercensal Population Estimate   
...         ...            ...   ...                              ...   
3649      36123   Yates County  2017   Postcensal Population Estimate   
3650      36123   Yates County  2018   Postcensal Population Estimate   
3651      36123   Yates County  2019   Postcensal Population Estimate   
3652      36123   Yates County  2020   Postcensal Population Estimate   
3653      36123   Yates County  2021   Postcensal Population Estimate   

      Population  
0         287114  
1         288615  
2         290302  
3         289754  
4         289148  
...      

In [76]:
# Create list of unique FIPS Codes for use with land area dataframe in next step

fipscodes = []

for i in range(len(popdenscounty['FIPS Code'])):
    if popdenscounty.iat[i,0] in fipscodes:
        pass
    else:
        fipscodes.append(popdenscounty.iat[i,0])

print(fipscodes)


[36001, 36003, 36005, 36007, 36009, 36011, 36013, 36015, 36017, 36019, 36021, 36023, 36025, 36027, 36029, 36031, 36033, 36035, 36037, 36039, 36041, 36043, 36045, 36047, 36049, 36051, 36053, 36055, 36057, 36059, 36061, 36000, 36063, 36065, 36067, 36069, 36071, 36073, 36075, 36077, 36079, 36081, 36083, 36085, 36087, 36091, 36093, 36095, 36097, 36099, 36089, 36101, 36103, 36105, 36107, 36109, 36111, 36113, 36115, 36117, 36119, 36121, 36123]


In [77]:
# Priya's api/web data code - "./Resources/health.ny.gov_table02.csv"

In [78]:
# Read Health.ny.gov data into dataframe from CSV

landareacounty=pd.read_csv("./Resources/health.ny.gov_table02.csv")
landareacounty=pd.DataFrame(landareacounty)
print(landareacounty)

            County 2018 Population Estimate 2010 Census Population  \
0   New York State               19,542,209             19,378,102   
1    New York City                8,398,748              8,175,133   
2            Bronx                1,432,132              1,385,108   
3            Kings                2,582,830              2,504,700   
4         New York                1,628,701              1,585,873   
..             ...                      ...                    ...   
59      Washington                   61,197                 63,216   
60           Wayne                   90,064                 93,772   
61     Westchester                  967,612                949,113   
62         Wyoming                   40,085                 42,155   
63           Yates                   24,841                 25,348   

   2010 Land Area Square Miles 2018 Population Density  
0                    47,126.40                  414.68  
1                       302.65               

In [79]:
# Drop all columns from landareacounty except 'County' and '2010 Land Area Square Miles'
reducelandareacounty = landareacounty[['County','2010 Land Area Square Miles']]

# Then remove rows with New York City Summaries
reducelandareacounty = reducelandareacounty.drop([reducelandareacounty.index[1]])

# Then sort by county name
sortreducelandarea = reducelandareacounty.sort_values(by='County')

# reset index and Add FIPS Code as Column
sortreducelandarea = sortreducelandarea.reset_index(drop=True)

sortreducelandarea.insert(0, "FIPS Code", fipscodes)

In [80]:
# Remove commas from 2010 Land Area Square Miles values
cleanlandarea = []
for i in range(len(sortreducelandarea)):
    cleanlandarea.append(sortreducelandarea.iat[i,2].replace(',', ''))

sortreducelandarea.insert(3,'Land Area (Square Miles)', cleanlandarea)
sortreducelandarea = sortreducelandarea[['FIPS Code','County','Land Area (Square Miles)']]

# Add 2010 Land Area Square Miles data into popdenscounty dataframe
popdenscountyland = pd.merge(popdenscounty, sortreducelandarea, how='outer')

# Remove 'County' column
popdenscountyland = popdenscountyland[['FIPS Code','Geography','Year','Program Type','Population','Land Area (Square Miles)']]
print(popdenscountyland)

      FIPS Code      Geography  Year                     Program Type  \
0         36001  Albany County  1970  Intercensal Population Estimate   
1         36001  Albany County  1971  Intercensal Population Estimate   
2         36001  Albany County  1972  Intercensal Population Estimate   
3         36001  Albany County  1973  Intercensal Population Estimate   
4         36001  Albany County  1974  Intercensal Population Estimate   
...         ...            ...   ...                              ...   
3649      36123   Yates County  2017   Postcensal Population Estimate   
3650      36123   Yates County  2018   Postcensal Population Estimate   
3651      36123   Yates County  2019   Postcensal Population Estimate   
3652      36123   Yates County  2020   Postcensal Population Estimate   
3653      36123   Yates County  2021   Postcensal Population Estimate   

      Population Land Area (Square Miles)  
0         287114                    522.8  
1         288615                   

In [81]:
# Convert Population and Land Area values to integer
popdenscountyland['Population'] = popdenscountyland['Population'].astype(int)
popdenscountyland['Land Area (Square Miles)'] = popdenscountyland['Land Area (Square Miles)'].astype(float)


# Add Population Density column
popdenscolumn = []
for i in range(len(popdenscountyland)):
    pop = popdenscountyland.iat[i,4]
    dens = popdenscountyland.iat[i,5]
    popdens = pop / dens
    popdenscolumn.append(popdens)

popdenscountyland['Population Density (Population per Square Mile)'] = popdenscolumn

print(popdenscountyland)

      FIPS Code      Geography  Year                     Program Type  \
0         36001  Albany County  1970  Intercensal Population Estimate   
1         36001  Albany County  1971  Intercensal Population Estimate   
2         36001  Albany County  1972  Intercensal Population Estimate   
3         36001  Albany County  1973  Intercensal Population Estimate   
4         36001  Albany County  1974  Intercensal Population Estimate   
...         ...            ...   ...                              ...   
3649      36123   Yates County  2017   Postcensal Population Estimate   
3650      36123   Yates County  2018   Postcensal Population Estimate   
3651      36123   Yates County  2019   Postcensal Population Estimate   
3652      36123   Yates County  2020   Postcensal Population Estimate   
3653      36123   Yates County  2021   Postcensal Population Estimate   

      Population  Land Area (Square Miles)  \
0         287114                    522.80   
1         288615               

In [82]:
# Remove non-Census Base Population rows

censusonly_popdenscntyland = popdenscountyland[popdenscountyland['Program Type']=='Census Base Population'].reset_index(drop=True)

print(censusonly_popdenscntyland)

     FIPS Code      Geography  Year            Program Type  Population  \
0        36001  Albany County  1970  Census Base Population      286742   
1        36001  Albany County  1980  Census Base Population      285909   
2        36001  Albany County  1990  Census Base Population      292812   
3        36001  Albany County  2000  Census Base Population      294601   
4        36001  Albany County  2010  Census Base Population      304199   
..         ...            ...   ...                     ...         ...   
373      36123   Yates County  1980  Census Base Population       21459   
374      36123   Yates County  1990  Census Base Population       22810   
375      36123   Yates County  2000  Census Base Population       24591   
376      36123   Yates County  2010  Census Base Population       25364   
377      36123   Yates County  2020  Census Base Population       24774   

     Land Area (Square Miles)  Population Density (Population per Square Mile)  
0                 

In [83]:
# Add 1970 starting pop density column
popden1970 = []
pop1970 = []

for i in range(len(censusonly_popdenscntyland['Population Density (Population per Square Mile)'])):
    if censusonly_popdenscntyland.iat[i,2]==1970:
        currpopden = censusonly_popdenscntyland.iat[i,6]
        currpop = censusonly_popdenscntyland.iat[i,4]
        popden1970.append(currpopden.astype(float))
        pop1970.append(currpop.astype(int))
    else:
        popden1970.append(currpopden.astype(float))
        pop1970.append(currpop.astype(float))

censusonly_popdenscntyland['Starting pop den. 1970']=popden1970
censusonly_popdenscntyland['Starting pop 1970']=pop1970

print(censusonly_popdenscntyland)


     FIPS Code      Geography  Year            Program Type  Population  \
0        36001  Albany County  1970  Census Base Population      286742   
1        36001  Albany County  1980  Census Base Population      285909   
2        36001  Albany County  1990  Census Base Population      292812   
3        36001  Albany County  2000  Census Base Population      294601   
4        36001  Albany County  2010  Census Base Population      304199   
..         ...            ...   ...                     ...         ...   
373      36123   Yates County  1980  Census Base Population       21459   
374      36123   Yates County  1990  Census Base Population       22810   
375      36123   Yates County  2000  Census Base Population       24591   
376      36123   Yates County  2010  Census Base Population       25364   
377      36123   Yates County  2020  Census Base Population       24774   

     Land Area (Square Miles)  \
0                      522.80   
1                      522.80   


In [84]:
# Add 50-Year population growth column
popgrow50yr = []

for i in range(len(censusonly_popdenscntyland['Population Density (Population per Square Mile)'])):
    if censusonly_popdenscntyland.iat[i,2]!=2020:
        popgrow50yr.append('')
    else:
        popgrow50yr.append((censusonly_popdenscntyland.iat[i,4] - censusonly_popdenscntyland.iat[i,8]) / censusonly_popdenscntyland.iat[i,8])

censusonly_popdenscntyland['50-Year Population Growth']=popgrow50yr

# Add 10-Year population growth column
popgrow10yr = []

for i in range(len(censusonly_popdenscntyland['Population Density (Population per Square Mile)'])):
    if censusonly_popdenscntyland.iat[i,2]==1970:
        popgrow10yr.append('')
    else:
        popgrow10yr.append((censusonly_popdenscntyland.iat[i,4] - censusonly_popdenscntyland.iat[i-1,4]) / censusonly_popdenscntyland.iat[i-1,4])

censusonly_popdenscntyland['10-Year Population Growth']=popgrow10yr

In [85]:
print(censusonly_popdenscntyland)

     FIPS Code      Geography  Year            Program Type  Population  \
0        36001  Albany County  1970  Census Base Population      286742   
1        36001  Albany County  1980  Census Base Population      285909   
2        36001  Albany County  1990  Census Base Population      292812   
3        36001  Albany County  2000  Census Base Population      294601   
4        36001  Albany County  2010  Census Base Population      304199   
..         ...            ...   ...                     ...         ...   
373      36123   Yates County  1980  Census Base Population       21459   
374      36123   Yates County  1990  Census Base Population       22810   
375      36123   Yates County  2000  Census Base Population       24591   
376      36123   Yates County  2010  Census Base Population       25364   
377      36123   Yates County  2020  Census Base Population       24774   

     Land Area (Square Miles)  \
0                      522.80   
1                      522.80   


In [88]:
# Export CSV to Output Folder
censusonly_popdenscntyland.to_csv('./Output/Output.csv', index = False)

In [86]:
#PRIYA ADD Code

# Add lat and long for each county
url="https://api.geoapify.com/v1/geocode/search?"

counties=censusonly_popdenscntyland["Geography"]
longitude=[]
latitude=[]

for county in counties:
    params = {  
        "text":county,
        "state":"New York",
        "apiKey":geoapify_key    
        }
    response=requests.get(url,params=params).json()
    # print(response)
    longitude.append(response["features"][0]["properties"]["lon"])
    latitude.append(response["features"][0]["properties"]["lat"])
print(longitude)
print(latitude)


# popdenscountyland["Longitude"]=longitude   
# popdenscountyland["Latitude"]=latitude

#CURRENTZEE#

KeyboardInterrupt: 

In [ ]:
popdenscountyland["Longitude"]=longitude   
popdenscountyland["Latitude"]=latitude

In [ ]:
# land_location2=land_location.loc[land_location["Year"]==2020,:]
# land_location2.reset_index()

In [ ]:

# plt.scatter(land_location2['Longitude'],land_location2['Latitude'],c=land_location2['Population Density (person/sq.mi.)'])
# clb = plt.colorbar()
# plt.xlabel("Longitude")
# plt.ylabel("Latitude")
# clb.set_label("Population Density (person/sq.mi.)")
# plt.show()

In [ ]:
# land_location3=land_location.loc[land_location["Year"]==2010,:]
# land_location3.reset_index()
# prev_yr_pop=land_location3["Population"]
# current_yr_pop=land_location2["Population"]
# pop_growth_percentage=[]

# for pop in prev_yr_pop:
#     for pops in current_yr_pop:
#         population_growth_rate=((current_yr_pop-prev_yr_pop)/prev_yr_pop)*100
#         pop_growth_percentage.append(population_growth_rate)
# print(pop_growth_percentage)

In [ ]:

# land_location2


In [ ]:
# map_plot=land_location2.hvplot.points("Longitude","Latitude",geo=True,tiles="OSM",frame_width=700,frame_height=500, 
#                                     size="Population Density (person/sq.mi.)",color="Geography")

# map_plot